In [2]:
%matplotlib inline

In [3]:
# gs: gold standard
# in: input
gs_cst_left_fn = '/Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/test_UKF/100307/2_Rs_0_025/WMQL/wmqa_100307_cst.left.vtp'
in_cst_left_fn = '/Shared/johnsonhj/HDNI/ReferenceData/HCP_DATA/test_UKF/100307/1_Rs_default_0_02/WMQL/wmqa_100307_cst.left.vtp'

In [4]:
def ComputeBhattacharyyaCoeficient(baseline_bundle, sr_bundle):
    # read in each fiber bundle
    # gs: gold standard, sr: super-resolution reconstructed
    import vtk
    import numpy as np
    ##
    def ReturnDistributionInEachCoordinate(bundle):
        from scipy import stats
        numPoints = bundle.GetNumberOfPoints()
        points = bundle.GetPoints()
        x_arr = np.array([points.GetPoint(i)[0] for i in xrange(numPoints)])
        y_arr = np.array([points.GetPoint(i)[1] for i in xrange(numPoints)])
        z_arr = np.array([points.GetPoint(i)[2] for i in xrange(numPoints)])
        x = np.linspace(x_arr.min(), x_arr.max(), 100)
        y = np.linspace(y_arr.min(), y_arr.max(), 100)
        z = np.linspace(z_arr.min(), z_arr.max(), 100)
        kde_x = stats.gaussian_kde(x_arr)
        kde_y = stats.gaussian_kde(y_arr)
        kde_z = stats.gaussian_kde(z_arr)
        p_x = kde_x(x)
        p_y = kde_y(y)
        p_z = kde_z(z)
        return p_x, p_y, p_z
    ## read in each fiber bundle
    reader_gs = vtk.vtkXMLPolyDataReader()
    reader_gs.SetFileName(baseline_bundle)
    reader_gs.Update()
    gs = reader_gs.GetOutput()
    #
    reader_sr = vtk.vtkXMLPolyDataReader()
    reader_sr.SetFileName(sr_bundle)
    reader_sr.Update()
    sr = reader_sr.GetOutput()
    ## Use ksdensity to estimate probability density function for the sample data in each coordinate
    [p_gs_x, p_gs_y, p_gs_z] = ReturnDistributionInEachCoordinate(gs)
    [p_sr_x, p_sr_y, p_sr_z] = ReturnDistributionInEachCoordinate(sr)
    #
    BC = (1.0/3.0)*( np.sum(np.sqrt(p_gs_x * p_sr_x)) + np.sum(np.sqrt(p_gs_y * p_sr_y)) + np.sum(np.sqrt(p_gs_z * p_sr_z)) )
    return BC

In [7]:
bc = ComputeBhattacharyyaCoeficient(gs_cst_left_fn,in_cst_left_fn)
print(bc)

0.97080829719
